In [1]:
import time
import pygame as py
import copy
import math

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
def gravity(player_pos, height):
    
    velocity = height/4
    running = True
    
    # FPS set-up
    dt = 0
    clock = py.time.Clock()
    
    while player_pos.y < height-height/25 and running:

        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                running = False
                
        screen.fill("white")
        
        
        py.draw.circle(screen, "indigo", player_pos, height/25)
        
        keys = py.key.get_pressed()
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
            
            
        player_pos.y -= (velocity*dt) + (-25* dt**2)
        velocity = velocity + (-50*dt)
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate-
        # independent physics.
        dt = clock.tick(60) / 1000
        
    return(running)

In [9]:
# Basic Setup
py.init()
py.font.init()

# FPS set-up
dt = 0
clock = py.time.Clock()

# Screen Parameters
width = 1000
height = 700

# Creating Font
font = py.font.SysFont('timesnewroman.ttf', int(width/20))

# Sets Up Window
screen = py.display.set_mode((width, height))
game1 = True
player_pos = py.Vector2(width/2, height/2)

def Start():
    running = True

    # Displaying Text
    text = font.render('Turn Back', True, 'black')
    
    # Text
    py.display.set_caption('Colours')
    
    while running:
        
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                running = False
                
        screen.fill("white")
        screen.blit(text, (5*width/8, height/8))
        
        py.draw.circle(screen, "indigo", player_pos, height/25)
        
        keys = py.key.get_pressed()
        if keys[py.K_w]:
            player_pos.y -= 300 * dt
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
        
        if player_pos.y != height-height/25 and running:
            running = gravity(player_pos, height)
            
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate-
        # independent physics.
        dt = clock.tick(60) / 1000

    
if game1 == True:
    Start()
    game1 == False
    
py.quit()